# Exploratory Data Analysis

This notebook contains initial Exploratory Data Analysis code. This is typically the first step in a data analysis pipeline and it is fundamental to:
 - get acquainted with the use-case
 - understand the nature and format of the data
 - explore the information and noise contained in the data
 - get insights of challenges we may face when training

## Problem

This dataset contains a Monte Carlo simulation of $\rho^{\pm} \rightarrow \pi^{\pm} + \pi^0$ decays and the corresponding detector response. Specifically, the data report the measured response of **i) tracker** and **ii) calorimeter**, along with the true pyshical quantitites that generated those measurements.

<div class="alert alert-block alert-info">
This means that we expect one track per event, with mainly two energy blobs (clusters of cells) in the calorimeter.
</div>

The final **goal** is to associate the cell signals observed in the calorimeter to the track that caused those energy deposits.

## Method

The idea is to leverage a **point cloud** data representation to combine tracker and calorimeter information so to associate cell hits to the corresponding track. We will use a [**PointNet**](https://openaccess.thecvf.com/content_cvpr_2017/papers/Qi_PointNet_Deep_Learning_CVPR_2017_paper.pdf) model that is capable of handling this type of data, framed as a **semantic segmentation** approach. More precisely, this means that:
- we represent each hit in the detector as a point in the point cloud: x, y, z coordinates + additional features ("3+"-dimensional point)
- the **learning task** will be binary classification at hit level: for each cell the model learns whether its energy comes mostly from the track (class 1) or not (class 0)

## Data structure

<div class="alert alert-block alert-info">

This dataset is organized as follows:
 - for each event, we create a **sample** (i.e. point cloud)
 - each sample contains all hits in a cone around a track of the event, called **focal track**
     - the cone includes all hits within some $\Delta R$ distance of the track
     - if an event has multiple tracks, then we have more samples per event
     - since different samples have possibly different number of hits, **we pad all point clouds to ensure they have same size** (needed since the model requires inputs of same size)

</div>

In [1]:
import numpy as np
from pathlib import Path

REPO_BASEPATH = Path().cwd().parent
DATA_PATH = REPO_BASEPATH / "pnet_data/raw/rho_small.npz"

events = np.load(DATA_PATH)["feats"]

In [2]:
### dataset content and types

# n_samples and points_per_sample
print("Data format and shape:")
print(f"{type(events)}\t{events.shape=}\n\n")
# Note: structured numpy array --> columns accessible by name

# dataset columns
print(f"Column\tdtype")
for colname, coltype in events.dtype.descr:
    print(f"{colname}\t:{coltype}")

Data format and shape:
<class 'numpy.ndarray'>	events.shape=(325, 800)


Column	dtype
event_number	:<i4
cell_ID	:<i4
track_ID	:<i4
delta_R	:<f4
truth_cell_focal_fraction_energy	:<f4
truth_cell_non_focal_fraction_energy	:<f4
truth_cell_neutral_fraction_energy	:<f4
truth_cell_total_energy	:<f4
category	:|i1
track_num	:<i4
x	:<f4
y	:<f4
z	:<f4
distance	:<f4
normalized_x	:<f4
normalized_y	:<f4
normalized_z	:<f4
normalized_distance	:<f4
cell_sigma	:<f4
track_chi2_dof	:<f4
track_chi2_dof_cell_sigma	:<f4
cell_E	:<f4
normalized_cell_E	:<f4
track_pt	:<f4
normalized_track_pt	:<f4
track_pt_cell_E	:<f4
normalized_track_pt_cell_E	:<f4


In [3]:
# dataset statistics

n_samples, n_points_per_sample = events.shape

sample_event_ids = events['event_number'][:,0]
event_ids, event_ids_count = np.unique(sample_event_ids, return_counts=True)
n_events = len(event_ids)

print(f"{n_events=}")
print(f"{n_samples=}")
print(f"{n_points_per_sample=}")

n_events=314
n_samples=325
n_points_per_sample=800


In [36]:
# take one point per sample and check event_number; 
# if event_number is repeated it means we have more samples per that event_number
# we take just one point, since event_number repeated for each point otherwise
n_samples_per_event = np.unique(events['event_number'][:, 0], return_counts=True)

multiple_samples_event_ids = n_samples_per_event[0][n_samples_per_event[1]>1]
print("This events unexpectedly have more than one sample per event... --> TO CHECK")
multiple_samples_event_ids

This events unexpectedly have more than one sample per event... --> TO CHECK


array([826871, 827140, 827188, 827226, 827242, 828437], dtype=int32)

## Number of tracks and cells per event/sample

In [83]:
for _ in events.dtype.names:
    print(_)

event_number
cell_ID
track_ID
delta_R
truth_cell_focal_fraction_energy
truth_cell_non_focal_fraction_energy
truth_cell_neutral_fraction_energy
truth_cell_total_energy
category
track_num
x
y
z
distance
normalized_x
normalized_y
normalized_z
normalized_distance
cell_sigma
track_chi2_dof
track_chi2_dof_cell_sigma
cell_E
normalized_cell_E
track_pt
normalized_track_pt
track_pt_cell_E
normalized_track_pt_cell_E


### tracks 

In [96]:
import pandas as pd

events_df = pd.DataFrame(events.reshape(-1).T, columns=events.dtype.names)
sample_id_col = np.repeat(np.arange(events.shape[0]), events.shape[1])
events_df['sample_ID'] = sample_id_col

n_tracks_per_sample = events_df.groupby('sample_ID')['track_ID'].max() + 1
np.unique(n_tracks_per_sample, return_counts=True)

(array([1, 2, 3], dtype=int32), array([307,   3,  15]))

In [97]:
n_tracks_per_event = events_df.groupby('event_number')['track_ID'].max() + 1
np.unique(n_tracks_per_event, return_counts=True)

(array([0, 1, 2, 3], dtype=int32), array([  1, 307,   2,   5]))

### cells 

In [103]:
n_cells_per_sample = events_df[events_df['cell_ID'] != -1].groupby('sample_ID')['cell_ID'].count()
np.unique(n_cells_per_sample, return_counts=True)

(array([ 25,  26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  37,  38,
         39,  40,  41,  42,  45,  46,  47,  48,  49,  51,  53,  56,  61,
         65,  67,  69,  71,  72,  73,  74,  76,  77,  79,  80,  81,  82,
         84,  87,  88,  89,  90,  91,  92,  95,  98, 100, 101, 107, 108,
        109, 110, 111, 112, 113, 118, 119, 120, 122, 123, 125, 126, 127,
        130, 131, 133, 135, 136, 137, 139, 141, 142, 143, 144, 145, 146,
        148, 149, 150, 153, 154, 155, 156, 157, 158, 159, 161, 162, 164,
        165, 166, 170, 174, 176, 177, 178, 181, 182, 184, 185, 186, 187,
        189, 190, 191, 194, 195, 196, 197, 198, 199, 201, 202, 208, 209,
        213, 216, 217, 218, 219, 221, 224, 225, 229, 230, 231, 233, 234,
        235, 236, 237, 239, 240, 242, 246, 247, 248, 249, 250, 251, 252,
        253, 254, 255, 256, 258, 259, 260, 262, 263, 264, 265, 268, 270,
        271, 276, 277, 278, 280, 289, 291, 293, 296, 298, 299, 300, 301,
        310, 314, 320, 323, 330, 331, 334, 338, 339

In [104]:
n_cells_per_event =  events_df[events_df['cell_ID'] != -1].groupby('event_number')['cell_ID'].count()
np.unique(n_cells_per_event, return_counts=True)

(array([  25,   26,   27,   28,   29,   30,   31,   32,   33,   34,   35,
          37,   38,   39,   40,   41,   42,   45,   46,   47,   48,   49,
          51,   53,   56,   61,   65,   67,   69,   71,   72,   73,   74,
          76,   77,   79,   80,   81,   82,   84,   87,   88,   89,   90,
          91,   92,   95,   98,  100,  101,  107,  108,  109,  110,  111,
         112,  113,  118,  120,  122,  123,  125,  126,  127,  130,  131,
         133,  135,  136,  137,  139,  141,  142,  143,  144,  145,  146,
         148,  149,  150,  153,  154,  155,  156,  157,  158,  159,  161,
         162,  164,  165,  166,  170,  174,  176,  177,  178,  181,  182,
         184,  185,  186,  187,  189,  190,  191,  194,  195,  196,  197,
         198,  199,  201,  202,  208,  209,  213,  216,  218,  219,  221,
         224,  225,  229,  230,  233,  234,  235,  236,  237,  239,  240,
         242,  246,  247,  248,  249,  250,  251,  252,  254,  255,  256,
         258,  259,  260,  262,  263, 

UP TO HERE

In [86]:
dtype = [('event_number', '<i4'), ('track_num', '<i4'), ('category', 'i1')]
    
# Create the data
event_num = np.array([[1, 1, 1, 1],
                      [1, 1, 1, 1],
                      [2, 2, 2, 2],
                      [3, 3, 3, 3]])
track_num = np.array([np.array([2, 2, 2, 2]), 
                     np.array([2, 2, 2, 2]), 
                     np.array([1, 1, 1, 1]), 
                     np.array([1, 1, 0, -1])])

category = np.array([np.array([1, 1, 0, -1]),
                    np.array([1, 0, 0, -1]),
                    np.array([1, 1, 1, 0]),
                    np.array([1, 1, 0, -1])])

# Initialize empty structured array
test_arr = np.zeros((4, 4), dtype=dtype)

# Fill the array
test_arr['event_number'] = event_num
test_arr['track_num'] = track_num
test_arr['category'] = category

#np_to_pd(test_arr[0])
test_arr

array([[(1,  2,  1), (1,  2,  1), (1,  2,  0), (1,  2, -1)],
       [(1,  2,  1), (1,  2,  0), (1,  2,  0), (1,  2, -1)],
       [(2,  1,  1), (2,  1,  1), (2,  1,  1), (2,  1,  0)],
       [(3,  1,  1), (3,  1,  1), (3,  0,  0), (3, -1, -1)]],
      dtype=[('event_number', '<i4'), ('track_num', '<i4'), ('category', 'i1')])

In [89]:
test_arr.reshape(-1)

array([(1,  2,  1), (1,  2,  1), (1,  2,  0), (1,  2, -1), (1,  2,  1),
       (1,  2,  0), (1,  2,  0), (1,  2, -1), (2,  1,  1), (2,  1,  1),
       (2,  1,  1), (2,  1,  0), (3,  1,  1), (3,  1,  1), (3,  0,  0),
       (3, -1, -1)],
      dtype=[('event_number', '<i4'), ('track_num', '<i4'), ('category', 'i1')])

In [53]:
test_arr[test_arr['event_number']==1].shape

(8,)

In [50]:
np_to_pd(test_arr[2])

,event_number,track_num,category
0,2,1,1
1,2,1,1
2,2,1,1
3,2,1,0


In [28]:
np.unique(test_arr[["track_num"]][:,], return_counts=True)

(array([(-1,), ( 0,), ( 1,)],
       dtype={'names': ['track_num'], 'formats': ['<i4'], 'offsets': [0], 'itemsize': 5}),
 array([ 1,  1, 10]))

In [30]:
np.unique(test_arr[["track_num", "category"]], return_counts=True)

(array([(-1, -1), ( 0,  0), ( 1, -1), ( 1,  0), ( 1,  1)],
       dtype=[('track_num', '<i4'), ('category', 'i1')]),
 array([1, 1, 1, 2, 7]))

In [19]:
np.unique(events[["track_num", "category"]][:, ], return_counts=True)

(array([(-1, 1), ( 0, 0), ( 1, 2)],
       dtype={'names': ['track_num', 'category'], 'formats': ['<i4', 'i1'], 'offsets': [33, 32], 'itemsize': 105}),
 array([151, 162,  12]))

In [6]:
events['track_num'][:,0])

SyntaxError: unmatched ')' (1309884531.py, line 1)

In [ ]:
sample_event.dtype

# Debug

## track_num and multi-track events 

In [ ]:
multiple_samples_event_ids = n_samples_per_event[0][n_samples_per_event[1]>1]

for _ in multiple_samples_event_ids:
    mask_event = events['event_number'] == multiple_samples_event_ids[0]
    mask_category = events['category'] == -1

    intersect = (mask_event * mask_category).sum()
    print(_, intersect)

# all zeros mean no padding points: this is because padding points have event_number==-1

In [61]:
# now inspect multi-track events

import pandas as pd

event_id = multiple_samples_event_ids[0]

sample_event = events[events['event_number']==event_id]
column_names = events.dtype.names


def np_to_pd(arr):
    column_names = arr.dtype.names
    
    data = np.array([arr[field] for field in column_names]).T
        
    # Create DataFrame with proper column names
    df = pd.DataFrame(data, columns=column_names)#.sort_values(by='category', ascending=False).reset_index(drop=True)
    return df


print(f"{event_id}\t", np.unique(sample_event['track_num'], return_counts=True))
df = np_to_pd(sample_event)
df.head()

826871	 (array([-1,  0,  1,  2], dtype=int32), array([355,  21,  21,  21]))


,event_number,cell_ID,track_ID,delta_R,truth_cell_focal_fraction_energy,truth_cell_non_focal_fraction_energy,truth_cell_neutral_fraction_energy,truth_cell_total_energy,category,track_num,...,normalized_distance,cell_sigma,track_chi2_dof,track_chi2_dof_cell_sigma,cell_E,normalized_cell_E,track_pt,normalized_track_pt,track_pt_cell_E,normalized_track_pt_cell_E
0,826871.0,-1.0,0.0,0.000041,-1.0,-1.0,-1.0,-1.0,0.0,0.0,...,0.0,-1.0,0.385282,0.385282,-1.0,-1.0,58.895405,0.557799,58.895405,0.557799
1,826871.0,-1.0,0.0,0.000000,-1.0,-1.0,-1.0,-1.0,0.0,0.0,...,0.0,-1.0,0.385282,0.385282,-1.0,-1.0,58.895405,0.557799,58.895405,0.557799
2,826871.0,-1.0,0.0,0.000173,-1.0,-1.0,-1.0,-1.0,0.0,0.0,...,0.0,-1.0,0.385282,0.385282,-1.0,-1.0,58.895405,0.557799,58.895405,0.557799
3,826871.0,-1.0,0.0,0.000326,-1.0,-1.0,-1.0,-1.0,0.0,0.0,...,0.0,-1.0,0.385282,0.385282,-1.0,-1.0,58.895405,0.557799,58.895405,0.557799
4,826871.0,-1.0,0.0,0.000471,-1.0,-1.0,-1.0,-1.0,0.0,0.0,...,0.0,-1.0,0.385282,0.385282,-1.0,-1.0,58.895405,0.557799,58.895405,0.557799


What is 'track_num'?

`it encodes track points: 0 (focus), N (unfocus track id), -1 (cell points or padding)`

--> to know this you should look at 'track_num' along with 'category'
--> it seems same information of 'track_ID', although values are not equal

Checks are below


In [69]:
cols = ['event_number', 'cell_ID', 'track_ID', 'track_num', 'category']
print(cols)

for i, _ in enumerate(sample_event[cols]):
    print(i, _)

['event_number', 'cell_ID', 'track_ID', 'track_num', 'category']
0 (826871, -1, 0, 0, 0)
1 (826871, -1, 0, 0, 0)
2 (826871, -1, 0, 0, 0)
3 (826871, -1, 0, 0, 0)
4 (826871, -1, 0, 0, 0)
5 (826871, -1, 0, 0, 0)
6 (826871, -1, 0, 0, 0)
7 (826871, -1, 2, 1, 2)
8 (826871, -1, 2, 1, 2)
9 (826871, -1, 2, 1, 2)
10 (826871, -1, 2, 1, 2)
11 (826871, -1, 2, 1, 2)
12 (826871, -1, 2, 1, 2)
13 (826871, -1, 2, 1, 2)
14 (826871, -1, 1, 2, 2)
15 (826871, -1, 1, 2, 2)
16 (826871, -1, 1, 2, 2)
17 (826871, -1, 1, 2, 2)
18 (826871, -1, 1, 2, 2)
19 (826871, -1, 1, 2, 2)
20 (826871, -1, 1, 2, 2)
21 (826871, 751063814, -1, -1, 1)
22 (826871, 752887558, -1, -1, 1)
23 (826871, 751063812, -1, -1, 1)
24 (826871, 752887556, -1, -1, 1)
25 (826871, 751063302, -1, -1, 1)
26 (826871, 751064326, -1, -1, 1)
27 (826871, 751063300, -1, -1, 1)
28 (826871, 749995072, -1, -1, 1)
29 (826871, 751063816, -1, -1, 1)
30 (826871, 751064324, -1, -1, 1)
31 (826871, 752887560, -1, -1, 1)
32 (826871, 807796736, -1, -1, 1)
33 (826871, 

In [81]:
# unfortunately we cannot subset events based on event_number since we leave out padding points
# --> we need to retrieve index of samples corresponding to event_number

for i, _ in enumerate(events):
    if _['event_number'][0]==event_id:
        print(i, _['event_number'][0])

44 826871
45 826871
46 826871


In [80]:
# this is just one sample! (800 points)
print(cols)

for i, _ in enumerate(events[cols][44,:]):
    print(i, _)

['event_number', 'cell_ID', 'track_ID', 'track_num', 'category']
0 (826871, -1, 0, 0, 0)
1 (826871, -1, 0, 0, 0)
2 (826871, -1, 0, 0, 0)
3 (826871, -1, 0, 0, 0)
4 (826871, -1, 0, 0, 0)
5 (826871, -1, 0, 0, 0)
6 (826871, -1, 0, 0, 0)
7 (826871, -1, 2, 1, 2)
8 (826871, -1, 2, 1, 2)
9 (826871, -1, 2, 1, 2)
10 (826871, -1, 2, 1, 2)
11 (826871, -1, 2, 1, 2)
12 (826871, -1, 2, 1, 2)
13 (826871, -1, 2, 1, 2)
14 (826871, -1, 1, 2, 2)
15 (826871, -1, 1, 2, 2)
16 (826871, -1, 1, 2, 2)
17 (826871, -1, 1, 2, 2)
18 (826871, -1, 1, 2, 2)
19 (826871, -1, 1, 2, 2)
20 (826871, -1, 1, 2, 2)
21 (826871, 751063814, -1, -1, 1)
22 (826871, 752887558, -1, -1, 1)
23 (826871, 751063812, -1, -1, 1)
24 (826871, 752887556, -1, -1, 1)
25 (826871, 751063302, -1, -1, 1)
26 (826871, 751064326, -1, -1, 1)
27 (826871, 751063300, -1, -1, 1)
28 (826871, 749995072, -1, -1, 1)
29 (826871, 751063816, -1, -1, 1)
30 (826871, 751064324, -1, -1, 1)
31 (826871, 752887560, -1, -1, 1)
32 (826871, 807796736, -1, -1, 1)
33 (826871, 

<div class="alert alert-block alert-warning">

Several questions:
 - why the choice for event_number?
 - do we need track_num as we already have same info in track_ID?

</div>